In [1]:
import preprocessing
import pandas as pd
import numpy as np

In [2]:
titles_file = 'docs_titles.tsv'
train_file = 'train_groups.csv'
test_file = 'test_groups.csv'
w2v_fpath = "all.norm-sz100-w10-cb0-it1-min100.w2v"

In [3]:
w2v = preprocessing.word2vec_model(w2v_fpath)

In [4]:
import pickle
with open('doc_to_title_nstemmed.pickle', 'rb') as handle:
    doc_to_title = pickle.load(handle)

In [5]:
doc_to_title_nstemmed = preprocessing.get_page_dict(titles_file, stemmer_I=False)

In [6]:
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.ensemble import ExtraTreesClassifier, BaggingClassifier, GradientBoostingClassifier, IsolationForest, RandomForestClassifier
from sklearn.svm import SVC
from sklearn.svm import OneClassSVM
from sklearn.linear_model import LogisticRegression, LinearRegression
import xgboost as xgb

In [7]:
f_dict = {}

In [8]:
clf = LinearRegression()

In [9]:
th, f = preprocessing.train_classifier(clf,  doc_to_title, doc_to_title_nstemmed, w2v, train_file, scaler_I='minmax')
f_dict['lin_reg'] = f

preparing tf-idf matrices....



calculating scores....

threshold: 0.33333333333333337 
f1: 0.7278605629009791


In [ ]:
lin_reg = preprocessing.predict_test(clf, th, doc_to_title, doc_to_title_nstemmed, w2v, train_file, test_file)

In [ ]:
clf_base = xgb.XGBClassifier(n_estimators=200, learning_rate=0.05, n_jobs=1)
clf = BaggingClassifier(base_estimator=clf_base, n_estimators=100, n_jobs=-1, max_features=0.7) 

In [ ]:
th, f = preprocessing.train_classifier(clf,  doc_to_title, doc_to_title_nstemmed, w2v, train_file)
f_dict['xgboost-bagging'] = f

In [ ]:
xgboost_bagging = preprocessing.predict_test(clf, th, doc_to_title, doc_to_title_nstemmed, w2v, train_file, test_file)

In [10]:
clf = GradientBoostingClassifier()

In [11]:
th, f = preprocessing.train_classifier(clf,  doc_to_title, doc_to_title_nstemmed, w2v, train_file)
f_dict['xgboost'] = f

preparing tf-idf matrices....



calculating scores....

threshold: 0.33333333333333337 
f1: 0.7392570734172642


In [ ]:
xgboost = preprocessing.predict_test(clf, th, doc_to_title, doc_to_title_nstemmed, w2v, train_file, test_file)

In [ ]:
clf = RandomForestClassifier(n_estimators=1000, random_state=111)

In [ ]:
th, f = preprocessing.train_classifier(clf,  doc_to_title, doc_to_title_nstemmed, w2v, train_file, scaler_I='minmax')
f_dict['random_forest'] = f

In [ ]:
random_forest = preprocessing.predict_test(clf, th, doc_to_title, doc_to_title_nstemmed, w2v, train_file, test_file, scaler_I='minmax')

In [ ]:
total_res = pd.read_csv(test_file)
total_res['target'] = xgboost_bagging*1
total_res[['pair_id', 'target']].to_csv('bag_boost_nmf_plus_word2vec_l.csv', index=None)

In [ ]:
total_res = pd.DataFrame({
              'xgboost-bagging': xgboost_bagging,
              'random_forest': random_forest,
              'lin_reg': lin_reg, 
              'logistic': logistic})

In [ ]:
total_res = total_res.applymap(lambda x: 1 if x else -1)

In [ ]:
res = (total_res.sum(axis=1) > 0) * 1

In [ ]:
total_res = pd.read_csv(test_file)
total_res['target'] = res

In [ ]:
total_res[['pair_id', 'target']].to_csv('lets_try_many_clf.csv', index=None)